In [1]:
# load libraries
import datasets
import re

In [2]:
# load video data
video_data = datasets.Dataset.load_from_disk("../data/videos/")
descriptions = video_data['description']
titles = video_data['title']

In [17]:
# clean descriptions
des = descriptions[5638]
des

' The death toll from a capsized fishing boat carrying migrants off the coast of southern Greece has risen to 78 the Greek coast guard has announced. A large-scale search and rescue operation...'

In [21]:
video_data['channel'][1]

'Al-Jazeera-English'

In [22]:
# maybe not sub all with "" but sub e.g. Fox and CNN with an Unk token or sth less specific
def clean_text(d:dict):
    # TODO: only take out CNN if channel == CNN ...
    c_ac:str = ''
    match d['channel']:
        case 'Fox-News':
           c_ac = 'Fox'
        case 'CNN-News':
            c_ac = 'CNN'
        case 'BBC-News':
            c_ac = 'BBC'
        case 'DW-News':
            c_ac = 'DW'
        case 'CCTV-Video-News-Agency':
            c_ac = 'CCTV News Agency'
        case 'Al-Jazeera-English':
            c_ac = 'Al Jazeera'
    title_clean = re.sub(
        '(Subscribe to)|'
        '((http).*(Watch))|'
        '((http).*(\.\.\.))|'
        '(News Agency)|'
        f'{c_ac}(#foxnews)|(#fox)|(#ingrahamangle)|'
        '(\\\\u0026 Friends Weekend)|'
        '(\\\\n)|(\\\\r)', '', d["title"])
    description_clean = re.sub(
        '(Subscribe to)|'
        '((http).*(Watch))|'  # complete youtube links
        '((http).*(\.\.\.))|'  # cut off links
        '(Newsfeed)|(News Agency)|'
        f'{c_ac}(#foxnews)|(#fox)|(#ingrahamangle)|'  # cha
        '(\\\\u0026 Friends Weekend)|'
        '(\\\\n)|(\\\\r)|'
        '(\| The Stream)', '', d["description"])
    return {"text_clean": f"{title_clean}. {description_clean}"}

In [23]:
# TODO: assign values or create new dataset
# add values to new column
cleaned_dataset = video_data.map(clean_text)

Map:   0%|          | 0/7908 [00:00<?, ? examples/s]

In [24]:
datasets.load_from_disk("../data/videos/")["title"]

["Twenty years on John Bolton is still defending the US's Iraq War | UpFront",
 'Paul Bremer on Iraq occupation mistakes and accountability | UpFront',
 'Indian opposition leader Rahul Gandhi disqualified as MP | Al Jazeera Newsfeed',
 'India’s Rahul Gandhi disqualified from parliament',
 'TikTok CEO Shou Zi Chew faces off with US legislators in hearing',
 'DR Congo: Displaced people with disabilities in need of help',
 'Ramadan in Gaza: Many fear another war during holy month',
 'The silent killer: Tuberculosis infections are on the rise',
 'Pakistan negotiates with IMF to unlock $1.1bn bailout',
 'Colombia president orders military operations against rebels',
 'Photographer captures beauty of Palestinian environment | Al Jazeera Newsfeed',
 'Final public hearing of UN probe on Israeli rights abuses',
 'Hotel Rwanda hero Paul Rusesabagina’s prison sentence commuted',
 'India arrests more than 100 people in manhunt for Sikh separatist',
 'Taiwan accuses China of involvement in Honduras

In [25]:
cleaned_dataset.save_to_disk("../data/videos_", max_shard_size="100MB")

Saving the dataset (0/1 shards):   0%|          | 0/7908 [00:00<?, ? examples/s]

In [ ]:
# was auch ginge wäre: du erstellst diene list of pre_processed_titles mit dem for loop
# und dann machst du dataset.add_column wie folgt
# dataset_with_length.add_column("cleaned_title", pre_processed_titles)

In [36]:
datasets.load_from_disk('../data/videos')['text_clean'][1232]

'Russia claims full control of Bakhmut in eastern Ukraine |  Newsfeed.  Russian mercenary Wagner Group claims to have fully captured the eastern Ukrainian city of Bakhmut but Ukrainian President Volodymyr Zelenskyy claims his troops are still there. Subscribe...'